In [77]:
import pandas as pd
import json
import re
import requests
import numpy as np

In [2]:
def request_data(call_url):
    r = requests.get(call_url)
    return json.loads(r.text)['data']

In [48]:
def get_trade_data(df, n=5, direction='exp'):
#     n = 5
    n_big_dict = {}
    if direction=='exp':
        trade_code = 'Exporter'
        print('Getting exports')
    else:
        trade_code = 'Importer'
        print('Getting imports')
    for cnt in df['id']:
    #     print(cnt, df.loc[df['id']==cnt, 'name'])
        
        html = r'https://oec.world/olap-proxy/data?cube=trade_i_baci_a_92&'+ trade_code + '+Country=' +cnt+ '&drilldowns=HS4&measures=Trade+Value&parents=true&Year=2018&sparse=false&locale=en&q=Trade Value'
    #     print(html)
        r2 = request_data(html)
        try:
            n_big_dict[ df.loc[df['id']==cnt, 'name'].values[0]] = pd.DataFrame(r2).sort_values('Trade Value', ascending=False)[['HS4', 'Section', 'Trade Value']].head(n)
        except:
            print(df.loc[df['id']==cnt, 'name'].values[0], 'not found')
    trade_df = pd.concat(n_big_dict).reset_index()
    trade_df.drop('level_1', axis=1, inplace=True)
    trade_df.rename(columns={'level_0': 'Country'}, inplace=True)
    return trade_df

### This part assumes you've downloaded the year_origin_destination_hs07_4 file and a few other files (apparent from the code below) from [oec](https://legacy.oec.world/en/resources/data/). If you would rather query the data directly from the website scroll down a bit

In [23]:
df_t = pd.read_csv('year_origin_destination_hs07_4.tsv.bz2')

In [ ]:
df_t = df_t.loc[44048806:] # Found by manual bisection method - gives data for 2017
df_t.reset_index(drop=True, inplace=True)
df_t = df_t['year\torigin\tdest\ths07\texport_val\timport_val'].apply(lambda x: re.sub('\s+',' ', x))
df_t = df_t.str.split(' ', expand=True)

In [25]:
df_t.columns = ['year', 'origin', 'dest', 'hs07', 'export_val', 'import_val']
df_t.loc[df_t['export_val']=='NULL', 'export_val'] = 0
df_t.loc[df_t['import_val']=='NULL', 'import_val'] = 0

In [26]:
df_t.head()

,year,origin,dest,hs07,export_val,import_val
0,2017,ago,bdi,8708,0,1546.00
1,2017,ago,cmr,0303,5984940.96,0
2,2017,ago,cmr,2520,33745.68,0
3,2017,ago,cmr,2523,346936.13,0
4,2017,ago,cmr,2710,0,7358.00


In [57]:
df_t['export_val'] = df_t['export_val'].astype(float)
df_t['import_val'] = df_t['import_val'].astype(float)

### Load product codes: 

In [30]:
df_p = pd.read_csv('products_hs_07.tsv.bz2', sep='delimiter')

C:\Users\GU216ZX\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [32]:
df_p.columns

Index(['"id"\t"hs07"\t"name"'], dtype='object')

In [36]:
df_p = df_p['"id"\t"hs07"\t"name"'].str.split('\t', expand=True)

In [47]:
df_p.head()

,0,1,2
0,"""010101""","""0101""","""Live horses, asses, mules and hinnies."""
1,"""01010110""","""010110""","""Live horses/asses/mules/hinnies: pure-bred br..."
2,"""01010190""","""010190""","""Live horses/asses/mules/hinnies other than pu..."
3,"""010102""","""0102""","""Live bovine animals."""
4,"""01010210""","""010210""","""Live bovine animals: pure-bred breeding animals"""


In [48]:
df_p.columns= ['id', 'hs07', 'name']

In [49]:
df_p.loc[df_p['hs07']=="2523", 'name'].values

array(['"Portland cement, aluminous cement, slag cement, supersulphate cement and similar hydraulic cements, whether or not coloured or in the form of clinkers."'],
      dtype=object)

### Use the product codes to update the trade dataframe with descriptions

In [64]:
df_t = df_t.merge(df_p[['hs07', 'name']], on='hs07')

###  Next load the country codes and add these to the party

In [61]:
df = pd.read_csv('country_names.tsv.bz2')

In [8]:
df.columns

Index(['id\t"id_3char"\t"name"'], dtype='object')

In [62]:
df = df['id\t"id_3char"\t"name"'].apply(lambda x: re.sub('\s+',' ', x)).str.split('\"', expand=True)[[0, 1, 3]]

In [63]:
df.rename(columns={0: 'id', 1: 'id_3char', 3: 'name'}, inplace=True)

In [ ]:
df['id'] = df['id'].str.strip()
df.loc[df['name'].str.contains('Tai'), ['id', 'id_3char', 'name']] = ['asxxb', 'xxb', 'Chinese Taipei']

In [74]:
df.to_csv('country_names.csv', index=False)

In [64]:
df.head()

,id,id_3char,name
0,afago,ago,Angola
1,afbdi,bdi,Burundi
2,afben,ben,Benin
3,afbfa,bfa,Burkina Faso
4,afbwa,bwa,Botswana


### Add the country names to the dataframe with the product information

In [79]:
df_tt = df_t
for c in zip(df[['id_3char', 'name']].values):
#     print(c[0][0], c[0][1])
    df_tt.loc[df_tt['origin']==c[0][0], 'origin'] = c[0][1]
    df_tt.loc[df_tt['dest']==c[0][0], 'dest'] = c[0][1]

In [80]:
df_tt.head()

,year,origin,dest,hs07,export_val,import_val,name
0,2017,Angola,Burundi,8708,0.0,1546.0,"""Parts and accessories of the motor vehicles o..."
1,2017,Angola,Republic of the Congo,8708,46166.0,155802.0,"""Parts and accessories of the motor vehicles o..."
2,2017,Angola,Algeria,8708,16864.0,0.0,"""Parts and accessories of the motor vehicles o..."
3,2017,Angola,Mozambique,8708,0.0,114912.0,"""Parts and accessories of the motor vehicles o..."
4,2017,Angola,Nigeria,8708,12558.0,0.0,"""Parts and accessories of the motor vehicles o..."


In [94]:
df_tt.sort_values('origin', inplace=True)
df_tt.reset_index(drop=True, inplace=True)

In [84]:
df_tt.to_csv('export_import_values_and_products.csv', index=False)

In [3]:
df_tt = pd.read_csv('export_import_values_and_products.csv')

In [6]:
df_tt.drop('Unnamed: 0', axis=1, inplace=True)

In [119]:
df_ev_sum = df_tt.groupby(['origin', 'hs07', 'name'])['export_val'].sum().reset_index()

In [120]:
df_ev_sum.head()

,origin,hs07,name,export_val
0,Afghanistan,1001,"""Wheat and meslin.""",0.0
1,Afghanistan,1003,"""Barley.""",0.0
2,Afghanistan,1005,"""Maize (corn).""",0.0
3,Afghanistan,1006,"""Rice.""",0.0
4,Afghanistan,1008,"""Buckwheat, millet and canary seed; other cere...",0.0


In [112]:
# df_ev_sum.sort_values(['origin', 'export_val'], ascending=False)

In [121]:
df_ev_sum = df_ev_sum.groupby('origin').apply(lambda x: x.sort_values('export_val', ascending=False).nlargest(5, 'export_val')).reset_index(drop=True)

In [122]:
df_ev_sum

,origin,hs07,name,export_val
0,Afghanistan,1301,"""Lac; natural gums, resins, gum-resins and ole...",1.021845e+08
1,Afghanistan,1211,"""Plants and parts of plants (including seeds a...",8.232279e+06
2,Afghanistan,5701,"""Carpets and other textile floor coverings, kn...",5.914243e+06
3,Afghanistan,5102,"""Fine or coarse animal hair, not carded or com...",5.221166e+06
4,Afghanistan,1207,"""Other oil seeds and oleaginous fruits, whethe...",4.162261e+06
5,Albania,6403,"""Footwear with outer soles of rubber, plastics...",3.457305e+08
6,Albania,6406,"""Parts of footwear (including uppers whether o...",1.907206e+08
7,Albania,2610,"""Chromium ores and concentrates.""",1.744078e+08
8,Albania,7202,"""Ferro-alloys.""",1.186383e+08
9,Albania,2709,"""Petroleum oils and oils obtained from bitumin...",1.175580e+08


## Downloading the data directly from the site
### Alternatively we could use the below to extract the, e.g., five biggest/import export products of each country of interest...

In [75]:
df = pd.read_csv('country_names.csv')

In [67]:
exp_df = get_trade_data(df)

Getting exports
Botswana not found
Western Sahara not found
Lesotho not found
Mayotte not found
Namibia not found
Reunion not found
Swaziland not found
Antarctica not found
Bouvet Island not found
Heard Island and McDonald Islands not found
South Georgia South Sandwich Islands not found
Midway not found
Yemen Arab Republic not found
Democratic Yemen not found
Belgium not found
Channel Islands not found
Czechoslovakia not found
Democratic Republic of Germany not found
Federal Republic of Germany not found
Faroe Islands not found
Isle of Man not found
Kosovo not found
Liechtenstein not found
Luxembourg not found
Monaco not found
Serbia and Montenegro not found
Svalbard not found
USSR not found
Holy See (Vatican City) not found
Yugoslavia not found
Martinique not found
Netherland Antilles and Aruba not found
Pacific Island (US) not found
Panama Canal Zone not found
Puerto Rico not found
United States Minor Outlying Islands not found
Virgin Islands not found
Guadeloupe not found
French Gui

In [49]:
exp_df.head(10)

,Country,HS4,Section,Trade Value
0,Afghanistan,Grapes,Vegetable Products,2.366653e+08
1,Afghanistan,Insect Resins,Vegetable Products,1.036503e+08
2,Afghanistan,Gold,Precious Metals,9.391353e+07
3,Afghanistan,Coal Briquettes,Mineral Products,8.698190e+07
4,Afghanistan,Tropical Fruits,Vegetable Products,8.294726e+07
5,Albania,Leather Footwear,Footwear and Headwear,3.576351e+08
6,Albania,Footwear Parts,Footwear and Headwear,2.294938e+08
7,Albania,Crude Petroleum,Mineral Products,1.853576e+08
8,Albania,Non-Knit Men's Suits,Textiles,1.229422e+08
9,Albania,Ferroalloys,Metals,1.173662e+08


In [68]:
imp_df = get_trade_data(df, direction='imp')

Getting imports
Botswana not found
Western Sahara not found
Lesotho not found
Mayotte not found
Namibia not found
Reunion not found
Swaziland not found
Antarctica not found
Bouvet Island not found
Heard Island and McDonald Islands not found
South Georgia South Sandwich Islands not found
Midway not found
Yemen Arab Republic not found
Democratic Yemen not found
Belgium not found
Channel Islands not found
Czechoslovakia not found
Democratic Republic of Germany not found
Federal Republic of Germany not found
Faroe Islands not found
Isle of Man not found
Kosovo not found
Liechtenstein not found
Luxembourg not found
Monaco not found
Serbia and Montenegro not found
Svalbard not found
USSR not found
Holy See (Vatican City) not found
Yugoslavia not found
Netherlands Antilles not found
Martinique not found
Netherland Antilles and Aruba not found
Pacific Island (US) not found
Panama Canal Zone not found
Puerto Rico not found
United States Minor Outlying Islands not found
Virgin Islands not found


### Minor adjustments 

In [70]:
exp_df.loc[exp_df['Country'].str.contains('Cze'), 'Country'] = 'Czechia'
imp_df.loc[imp_df['Country'].str.contains('Cze'), 'Country'] = 'Czechia'

### Save as csv

In [73]:
exp_df.to_csv('five_biggest_exports.csv', index=False)
imp_df.to_csv('five_biggest_imports.csv', index=False)